In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003820657730102539
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 17.440128767718196
200000 26.190832066507678
300000 26.011374724381053
400000 25.74235864206337
500000 27.457175144311382
600000 25.258554821206527
700000 24.92303163740608
800000 25.028578539657165
900000 23.92380975721985
1000000 10.899019885175408
1100000 24.384333310242038
1200000 24.803136333397408
1300000 24.458014424359327
1400000 25.520015470497654
1500000 25.505596475717415
1600000 24.894552746147006
1700000 25.26669373401362
1800000 25.20694602770508
1900000 10.175083616119174
2000000 26.12338487595458
2100000 31.13177001545893
2200000 30.273182335093367
2300000 30.10466791378557
2400000 31.098778007637733
2500000 29.123449283535173
2600000 29.08425983658661
2700000 29.05881722565665
2800000 18.276422344653547
2900000 22.772494965099767
3000000 

29400000 24.641632706705458
29500000 24.105202904365598
29600000 12.37265544417542
29700000 23.939226020233626
29800000 24.60598806182489
29900000 24.668686743866427
30000000 22.606928031742854
30100000 13.234415552386501
30200000 24.673535866926155
30300000 24.21084509803594
30400000 23.43028942850405
30500000 22.68566119156641
30600000 21.312886140976538
30700000 16.280976960608513
30800000 15.002361750805406
30900000 21.385537430153903
31000000 22.25739697489955
31100000 23.002758900703476
31200000 23.649665666557112
31300000 24.530515628103803
31400000 15.360053044796699
31500000 20.943235208638
31600000 23.99162621755949
31700000 24.955288985822076
31800000 24.38106277113904
31900000 18.20909783721414
32000000 18.918673362512646
32100000 25.46420839607477
32200000 23.48418956016631
32300000 24.619388384037038
32400000 22.49865491699121
32500000 21.689339273449928
32600000 9.02207128184839
32700000 22.815564441523772
32800000 21.558048447480974
32900000 23.50574975792401
33000000 2

59200000 19.94549871997248
59300000 20.495120112692057
59400000 20.912796951266483
59500000 21.54342173821923
59600000 21.96212946228729
59700000 21.529654879209065
59800000 21.18929884328239
59900000 20.491603336180646
60000000 21.016229605742595
60100000 9.168212041173796
60200000 19.062037996713066
60300000 18.829909230829458
60400000 18.971579066987186
60500000 9.70570103203253
60600000 17.105503624800455
60700000 19.209453034223447
60800000 18.91195481990832
60900000 16.855930668377184
61000000 13.505489943256974
61100000 20.238740784813118
61200000 20.628222653847043
61300000 20.5452847341249
61400000 20.99606825228631
61500000 21.815844605962067
61600000 21.308352561161247
61700000 21.90839572476609
61800000 20.687680829763938
61900000 17.291016634643825
62000000 12.063600475814784
62100000 19.135398921283127
62200000 19.807173592521636
62300000 19.69642920222178
62400000 7.619995749479228
62500000 19.296834407280823
62600000 19.023581210054044
62700000 19.00537481253104
6280000

88900000 17.51326770736019
89000000 17.32646693362004
89100000 17.448516970588123
89200000 18.613041312405855
89300000 19.1689613634484
89400000 19.247237683568716
89500000 18.946948515406827
89600000 18.017255906628947
89700000 17.055535632939485
89800000 17.130519816659064
89900000 17.669917228746716
90000000 16.582334442320615
90100000 8.26900388020575
90200000 16.626886612108315
90300000 6.44681540267044
90400000 17.314710758371522
90500000 8.8676097545576
90600000 17.16029174544456
90700000 17.256506636294347
90800000 17.168176015134566
90900000 17.008050748671923
91000000 17.493028180252164
91100000 18.733169235855982
91200000 18.723306231845807
91300000 19.82237496328783
91400000 18.258282601153283
91500000 17.1195404551958
91600000 17.132711592523375
91700000 17.381502294155634
91800000 17.42073968518463
91900000 16.5593870130755
92000000 8.588389532093432
92100000 13.778379916006603
92200000 10.54065183572321
92300000 11.833163025645836
92400000 13.85126977469979
92500000 17.2

118000000 1.735204147139579
118100000 1.7321515259439118
118200000 1.7402010675404038
118300000 1.7243426165141662
118400000 1.715718487005959
118500000 1.7140363676790462
118600000 1.7221311857033073
118700000 1.7273371875902184
118800000 1.709181912839843
118900000 1.7094115225488544
119000000 1.7084462958771556
119100000 2.520451261040681
119200000 22.741105447135585
119300000 21.199241013042048
119400000 21.224914642291836
119500000 20.593365412733714
119600000 20.66237072102946
119700000 20.13263591703292
119800000 19.84393312523693
119900000 20.588715893423046
120000000 10.772975761234298
120100000 2.648694618765255
120200000 18.786294099682028
120300000 19.617058528756484
120400000 20.320784827355574
120500000 20.113976465203038
120600000 21.20901774847481
120700000 20.686281755574168
120800000 21.305337975324885
120900000 21.897690720318717
121000000 21.311425987007407
121100000 17.320805201337905
121200000 17.246249613224006
121300000 16.372673939695648
121400000 16.8911154629

...

233200000 2.269206480361283
233300000 2.065148839458323
233400000 2.0665473343303535
233500000 0.9674212886520226
233600000 0.5308067072638737
233700000 1.5274896694877709
233800000 2.4249236885237595
233900000 1.98953690596053
234000000 2.1917247311484385
234100000 2.0985336517741975
234200000 2.437116145684186
234300000 2.2058660567619537
234400000 1.9442572397187152
234500000 2.2416599432264994
234600000 4.200059454474366
234700000 1.8937306473200919
234800000 2.809118361124234
234900000 3.7566575008291077
235000000 2.6889101472337846
235100000 3.9143099032682214
235200000 1.986623690000471
235300000 2.1432644402210315
235400000 0.773670038374083
235500000 0.5572851785897514
235600000 2.1775231690356134
235700000 2.031896184644484
235800000 3.109133783811251
235900000 2.8632901381808225
236000000 2.7348583162278235
236100000 2.4251496287352574
236200000 3.1814390449489958
236300000 2.8169485035142534
236400000 3.4182340596526566
236500000 2.3731855165623923
236600000 1.8454140894638

261900000 3.083342684322172
262000000 3.122773534091025
262100000 3.071540915093114
262200000 3.111212896408369
262300000 3.1909575976948834
262400000 3.936785443185224
262500000 3.831617746598294
262600000 3.5205367214414682
262700000 3.173487852428165
262800000 3.2681355293480623
262900000 3.359806609735489
263000000 3.318854372395214
263100000 3.0972770282988753
263200000 3.5462334110127594
263300000 0.7665593390299877
263400000 0.7630682126407027
263500000 2.839338114449203
263600000 3.2566989650635683
263700000 3.2804134680370685
263800000 3.5206712969708738
263900000 3.2499158976579205
264000000 2.9866484022644593
264100000 3.361152103146553
264200000 3.3876123202557764
264300000 4.417970849024614
264400000 4.20516922409902
264500000 3.6478203177811905
264600000 3.4245855826764275
264700000 3.7566866799326237
264800000 3.862921055825753
264900000 3.560661771053487
265000000 3.204601872365246
265100000 3.182757896489948
265200000 0.7419593572327957
265300000 1.3237187056413193
265

290700000 2.003782768683705
290800000 2.2637576717806756
290900000 2.129116754757934
291000000 2.072900867195884
291100000 2.148681090377182
291200000 0.8208902721954121
291300000 0.5178535602858828
291400000 1.838635474851529
291500000 1.9574054062279098
291600000 2.096229159151475
291700000 2.135408171066582
291800000 2.2287590989037485
291900000 1.9789281357712878
292000000 2.1454411685047745
292100000 2.2355807462707658
292200000 2.566011721537229
292300000 2.1656697460885246
292400000 2.24557756046469
292500000 1.896639084114545
292600000 2.0806554332684875
292700000 2.0988857602273887
292800000 2.234020075107541
292900000 2.0032513273224435
293000000 1.7486637382005883
293100000 0.5031895830295119
293200000 0.498000553403149
293300000 2.3758534873674413
293400000 1.9846235002360906
293500000 2.0791652417435644
293600000 2.1052942079416814
293700000 2.1860807585377167
293800000 1.971609621393611
293900000 2.2278937251850004
294000000 2.153600059787876
294100000 2.4587082037620442


318500000 1.1630886571191814
318600000 1.3181263505104517
318700000 1.191916939665109
318800000 1.3313445825653636
318900000 1.2442852029398348
319000000 1.2014066458621322
319100000 1.1079028297125897
319200000 1.3210093693746525
319300000 1.1506245268746913
319400000 1.2098405903765228
319500000 1.1653446017696063
319600000 1.1089976713768839
319700000 1.3218783686059017
319800000 1.3517622359460426
319900000 1.1150287185768684
320000000 1.1733580800812244
320100000 1.1955370832119816
320200000 1.2312288446795938
320300000 1.262206382249524
320400000 1.2082919628283515
320500000 1.1776912569184004
320600000 1.2377302002264194
320700000 1.078246285916613
320800000 1.1340839448634088
320900000 1.0617074389713341
321000000 1.1172587455428409
321100000 1.1724146975828509
321200000 1.1137407211041153
321300000 1.0963822765238087
321400000 1.0528430701163412
321500000 0.9802788564266861
321600000 1.1366179499570297
321700000 1.0983388633848727
321800000 1.0577083605413204
321900000 1.19693

345500000 0.1834862167389426
345600000 0.14426052487252486
345700000 0.14173547736662626
345800000 0.13119989805830187
345900000 0.13802873898626697
346000000 0.1484727805709243
346100000 0.1364668910564416
346200000 0.13549990169981785
346300000 0.0967548715299368
346400000 0.17065763920850222
346500000 0.13726003966476522
346600000 0.1176170616570413
346700000 0.12370113467888368
346800000 0.10841046576747299
346900000 0.1077757544191877
347000000 0.11469069833194216
347100000 0.13830803908826245
347200000 0.12120205579753718
347300000 0.11309210666545232
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.07376947789366047
347500000 0.01874879030370712
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
type: [1, 41, 3, 1, 1] 123
cases of this type: 369
best so far: 0
type: [1, 123, 1, 1, 1] 1